# Proyecto Modulo 3


<a href="https://colab.research.google.com/github/ArturoG1z/Proyecto-Modulo-3-CyAD/blob/main/Notebook/proyecto_modulo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***PENDIENTE:*** Agregar descripción pequeña del problema del dataset y del objetivo


In [50]:
import pandas as pd
ruta = '/content/drive/MyDrive/Colab Notebooks/PROYECTO/data set con color.xlsx'
df = pd.read_excel(ruta)

In [ ]:
df.head()

In [51]:
df.shape

(747, 41)

### Limpieza de los datos

In [ ]:
# Se eliminan columas/atributos despreciables
columnas = df.columns[[0,3,6,7,31,32,35,36,37,38,39,40]]
#print(columnas)
df.drop(columnas, axis=1, inplace=True)

# Eliminando el ID de la materia en la columna 'Comunidad'
df['Comunidad'] = df['Comunidad'].str[5:]

# Separando la columna 1er P_Ponderación\semanas
df[['1erPonderacion', 'Semana_1erP']] = df['1er P_Ponderación\semanas'].str.split('\\', 1, expand=True)
df = df.drop('1er P_Ponderación\semanas', axis=1)

# Separando la columna 2oP_Ponderación\semanas
df[['2daPonderacion', 'Semana_2daP']] = df['2oP_Ponderación\semanas'].str.split('\\', 1, expand=True)
df = df.drop('2oP_Ponderación\semanas', axis=1)

# Reemplaza los valores NaN por 0
df.fillna(0, inplace=True)


df.tail()
#df.shape

In [60]:

# Cuenta cuántos 0 hay en la columna
numero_ceros_1er = df['1er P_Porcentaje aprobados'].value_counts()[0]
numero_ceros_2do = df['2oP_Porcentaje aprobados'].value_counts()[0]

# Imprime el número de 0
print(numero_ceros_1er)

# Imprime el número de 0
print(numero_ceros_2do)



360
362
